In [17]:
# Import needed libraries
import pandas as pd
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split

In [2]:
# Open the transposable elements data as a dataframe.
te_df = pd.read_table("../../../data/2018_06_12_te_enhancers_ml/test.tsv", header = None)

In [3]:
# Create a data frame to store all the transposable element locations
te_loc_df = pd.DataFrame(columns = ["chr", "start", "end"])
te_loc_df["chr"] = te_df.iloc[:,0]
te_loc_df["start"] = te_df.iloc[:,1]
te_loc_df["end"] = te_df.iloc[:,2]

In [4]:
# Delete repeats from the data frame and reindex.
te_new_df = te_loc_df.copy().drop_duplicates()
te_new_df.index = range(len(te_new_df["chr"])) 

In [5]:
# Get the set of all transcription factors as column labels.
col_set = set(te_df.iloc[:,8])

# Convert to a list.
col_list = list(col_set)  
    
# Add all the transcription factors as column labels for the new data frame.
for tf in col_list:
    te_new_df[tf] = 0
    
# Create a column for if enhancer overlaps transposable element
te_new_df["enhancer_actual"] = 0

In [6]:
# Iterate through the original dataframe
for row in te_df.itertuples():
    # Chromosome is now in index 1; start location in index 2; end location in index 3 of row.
    # Match chromosome, start location, and end location from the old and new data frames
    # and update the corresponding column of the transcription factor in the new data frame.
    te_new_df.loc[((te_new_df["chr"] == row[1]) & 
                (te_new_df["start"] == row[2]) &
                (te_new_df["end"] == row[3])), [row[9]]] += 1
    
    # Update the enhancer column as needed.
    if row[14] == "1":
        te_new_df.loc[((te_new_df["chr"] == row[1]) & 
                (te_new_df["start"] == row[2]) &
                (te_new_df["end"] == row[3])), "enhancer_actual"] = 1

In [7]:
# Set column for machine learning "y" vector as the last one, so all transcription factors plus
# three columsn for the locations.
end_index = len(col_set) + 3

# Set the machine learning input vector as all columns of transcription factors
x_df = te_new_df.copy().iloc[:,3:end_index]

# Set the machine learning prediction vector as the last column, which tells if enhancer is present
y_actual = te_new_df.copy().iloc[:,end_index]

In [8]:
# Create a random forest classifier model
rfc = RandomForestClassifier(n_estimators = 1000, n_jobs = -1)

In [9]:
# Perform 5-fold cross validation on the random forest model
cvs = cross_val_score(rfc, x_df, y_actual, cv = 5)

In [10]:
# Print the cross validation scores to a file.
cvs_df = pd.DataFrame(data = cvs, index = ["cvs 1", "cvs 2", "cvs 3", "cvs 4", "cvs 5"], columns = ["score"])
cvs_df.to_csv("../../../results/2018_06_14/local/cross_val_score.csv", sep = '\t', index = False)

In [11]:
# Create predictions using 5-fold cross validation to view incorrect predictions
y_pred = cross_val_predict(rfc, x_df, y_actual, cv = 5)

In [12]:
# Convert the prediction results to a dataframe.
predictions_df = pd.DataFrame(data = y_pred, columns = ["enhancer_predicted"])

In [13]:
# Create a dataframe to combine predictions with actual data
output_df = pd.DataFrame(te_new_df.copy()[["chr", "start", "end", "enhancer_actual"]])

# Copy over predictions and print to csv file
output_df["enhancer_predicted"] = predictions_df
output_df.to_csv("../../../results/2018_06_14/local/predictions.csv", sep = '\t')

In [14]:
# Create a confusion matrix to view results
cm_df = pd.DataFrame(metrics.confusion_matrix(y_actual, y_pred), index = ["actual_negative", "actual_positive"]
                    , columns = ["predicted_negative", "predicted_positive"])
cm_df.to_csv("../../../results/2018_06_14/local/confusion_matrix.csv", sep = '\t')

In [15]:
# Create a file to store metrics
metrics_file = open("../../../results/2018_06_14/local/metrics.txt", "w+")

In [16]:
# Print the f1 score to a file
metrics_file.write("Recall: " + str(metrics.recall_score(y_actual, y_pred)))
metrics_file.close()

In [21]:
# Predict probabilities
xtrain, xtest, ytrain, ytest = train_test_split(x_df, y_actual)
rfc.fit(xtrain, ytrain)
ypred = rfc.predict_proba(xtest)
print(ypred)

[[9.35337225e-01 6.46627753e-02]
 [1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00]
 [9.98221598e-01 1.77840164e-03]
 [1.00000000e+00 0.00000000e+00]
 [9.87342396e-01 1.26576036e-02]
 [9.85137480e-01 1.48625199e-02]
 [1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00]
 [9.99987654e-01 1.23456790e-05]
 [9.85137480e-01 1.48625199e-02]
 [9.87342396e-01 1.26576036e-02]
 [1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00]
 [9.85137480e-01 1.48625199e-02]
 [9.97728559e-01 2.27144076e-03]
 [1.00000000e+00 0.00000000e+00]
 [9.99407540e-01 5.92460317e-04]
 [9.87342396e-01 1.26576036e-02]
 [9.85137480e-01 1.48625199e-02]
 [9.87342396e-01 1.26576036e-02]
 [9.87342396e-01 1.26576036e-02]
 [1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00]
 [9.87342396e-01 1.26576036e-02]
 [1.00000000e+00 0.00000000e+00]
 [1.000000